In [1]:
from IPython.display import clear_output
from data_generator import DataGenerator
from medium_generator import MediumGenerator, disk_func, cosine_func
import numpy as np
from numpy.linalg import lstsq
from ngsolve import * 
from ngsolve.webgui import Draw

In [2]:
# main functions are in data_generator.py

# The domain discretization parameters can be made smaller 
# if higher accuracy is needed but takes significantly longer time to run
# The mesh size must be smaller if the wave number is higher. 
# The following configuration should be OK for wave_num less than 10

dg = DataGenerator(maxh = (0.1, 0.2)) 
medium = MediumGenerator(cosine_func) # it generates a medium permittivity q. There is another choice MediumGenerator(disk_func).

background_params = [{"type": 0, "x": 0.0, "y": 0.0, "r": 0.5, "v": 0.0}]
background_permittivity = medium.generate(background_params)

params  = [{"type": 0, "x": 0.6 ,"y": 0.4, "r": 0.3, "v": 1}, 
           {"type": 0, "x": 0.7, "y": -0.6, "r": 0.3, "v": 1},
           {"type": 0, "x": -0.8, "y": -0.3, "r": 0.3, "v": 1},
           {"type": 0, "x": -0.5, "y": 0.5, "r": 0.3, "v": 1}]

permittivity  = medium.generate(params) # it generates 4 bumps.

Draw(permittivity, dg.mesh)

Mesh generation took 0.1426902879611589 seconds


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

In [3]:
inc_kx, inc_ky = 6, 8 # wave number = 10
u_scat = dg.solve(inc_kx, inc_ky, permittivity) # scattered wave
Draw(u_scat, dg.mesh)

Solving took 0.16292239999165758 seconds


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {'Complex': {'phase': 0.0, 'sp…

BaseWebGuiScene

In [4]:
wave_num = 10
direction_angles = np.linspace(0, 2 * np.pi, 16, endpoint=False) # generate 16 directions of incident waves
cauchy_data = dg.generate_cauchy_data(wave_num, direction_angles, permittivity) # wave number = 10

print(cauchy_data[0].shape, cauchy_data[1].shape) # Dirichlet/Neumann data on 360 receivers

Solving took 0.14434957498451695 seconds
Solving took 0.13450905797071755 seconds
Solving took 0.13236802897881716 seconds
Solving took 0.13174756499938667 seconds
Solving took 0.13611812703311443 seconds
Solving took 0.12887622200651094 seconds
Solving took 0.13149415102088824 seconds
Solving took 0.1412923900061287 seconds
Solving took 0.13887087302282453 seconds
Solving took 0.14123806497082114 seconds
Solving took 0.1365637500421144 seconds
Solving took 0.132687019999139 seconds
Solving took 0.13259479799307883 seconds
Solving took 0.130798285943456 seconds
Solving took 0.13496567698894069 seconds
Solving took 0.1406139949685894 seconds
(16, 360) (16, 360)


In [5]:
"""
Now check Green's formula with the boundary integral. 

-Delta u_{tot, i} - k^2 q u_{tot, i} = 0
-Delta u_{tot, j} - k^2 q u_{tot, j} = 0

multiply 1st with u_{tot, j} and 2nd with u_{tot, i}

int_{partial D} ( partial_n u_{tot, i} ) u_{tot, j} - (partial_n u_{tot, j}) u_{tot, i} = 0.

or equivalently

int_{partial D} ( partial_n u_{scat, i} ) u_{scat, j} - (partial_n u_{scat, j}) u_{scat, i} = 0.

"""
wave_num = 10
i, j = 5, 12 

inc_kx = wave_num * np.cos(direction_angles[i])
inc_ky = wave_num * np.sin(direction_angles[i])

u_scat_i = dg.solve(inc_kx, inc_ky, permittivity)
u_inc_i = CF((exp(1j * inc_kx * x) * exp(1j * inc_ky * y)))

inc_kx = wave_num * np.cos(direction_angles[j])
inc_ky = wave_num * np.sin(direction_angles[j])

u_scat_j = dg.solve(inc_kx, inc_ky, permittivity)
u_inc_j = CF((exp(1j * inc_kx * x) * exp(1j * inc_ky * y)))

LHS = np.sum( cauchy_data[1][i, :] * cauchy_data[0][j, :] - cauchy_data[1][j, :] * cauchy_data[0][i, :] ) / cauchy_data[0].shape[1] * 2 * np.pi * dg.cauchy_radius

print(LHS)

Solving took 0.1486183870001696 seconds
Solving took 0.13708691799547523 seconds
(-2.917394460024623e-05+5.8244010892389306e-05j)
